In [1]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

from spektral.datasets import citation
from spektral.layers import GraphConv

# Load data
dataset = 'cora'
A, X, y, train_mask, val_mask, test_mask = citation.load_data(dataset)

# Parameters
channels = 16           # Number of channels in the first layer
N = X.shape[0]          # Number of nodes in the graph
F = X.shape[1]          # Original size of node features
n_classes = y.shape[1]  # Number of classes
dropout = 0.5           # Dropout rate for the features
l2_reg = 5e-4 / 2       # L2 regularization rate
learning_rate = 1e-2    # Learning rate
epochs = 200            # Number of training epochs
es_patience = 10        # Patience for early stopping

Loading cora dataset
Pre-processing node features


In [2]:
# Preprocessing operations
fltr = GraphConv.preprocess(A).astype('f4')
X = X.toarray()

# Model definition
X_in = Input(shape=(F, ))
fltr_in = Input((N, ), sparse=True)

dropout_1 = Dropout(dropout)(X_in)
graph_conv_1 = GraphConv(channels,
                         activation='relu',
                         kernel_regularizer=l2(l2_reg),
                         use_bias=False)([dropout_1, fltr_in])
dropout_2 = Dropout(dropout)(graph_conv_1)
graph_conv_2 = GraphConv(n_classes,
                         activation='softmax',
                         use_bias=False)([dropout_2, fltr_in])

# Build model
model = Model(inputs=[X_in, fltr_in], outputs=graph_conv_2)
optimizer = Adam(lr=learning_rate)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              weighted_metrics=['acc'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1433)]       0                                            
__________________________________________________________________________________________________
dropout (Dropout)               (None, 1433)         0           input_1[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 2708)]       0                                            
__________________________________________________________________________________________________
graph_conv (GraphConv)          (None, 16)           22928       dropout[0][0]                    
                                                                 input_2[0][0]                

In [3]:
# Train model
validation_data = ([X, fltr], y, val_mask)
model.fit([X, fltr],
          y,
          sample_weight=train_mask,
          epochs=epochs,
          batch_size=N,
          validation_data=validation_data,
          shuffle=False,  # Shuffling data means shuffling the whole graph
          callbacks=[
              EarlyStopping(patience=es_patience,  restore_best_weights=True)
          ])

# Evaluate model
print('Evaluating model.')
eval_results = model.evaluate([X, fltr],
                              y,
                              sample_weight=test_mask,
                              batch_size=N)
print('Done.\n'
      'Test loss: {}\n'
      'Test accuracy: {}'.format(*eval_results))

Epoch 1/200
1/1 [==============================] - 0s 305ms/step - loss: 0.1085 - acc: 0.1857 - val_loss: 0.3645 - val_acc: 0.2180
Epoch 2/200
1/1 [==============================] - 0s 169ms/step - loss: 0.1057 - acc: 0.2286 - val_loss: 0.3622 - val_acc: 0.3240
Epoch 3/200
1/1 [==============================] - 0s 166ms/step - loss: 0.1035 - acc: 0.5071 - val_loss: 0.3606 - val_acc: 0.3880
Epoch 4/200
1/1 [==============================] - 0s 169ms/step - loss: 0.1020 - acc: 0.5571 - val_loss: 0.3595 - val_acc: 0.4560
Epoch 5/200
1/1 [==============================] - 0s 167ms/step - loss: 0.1011 - acc: 0.6143 - val_loss: 0.3589 - val_acc: 0.4740
Epoch 6/200
1/1 [==============================] - 0s 168ms/step - loss: 0.1005 - acc: 0.6429 - val_loss: 0.3586 - val_acc: 0.4640
Epoch 7/200
1/1 [==============================] - 0s 168ms/step - loss: 0.1004 - acc: 0.6357 - val_loss: 0.3584 - val_acc: 0.4440
Epoch 8/200
1/1 [==============================] - 0s 164ms/step - loss: 0.1003 - a

1/1 [==============================] - 0s 162ms/step - loss: 0.0947 - acc: 0.6786 - val_loss: 0.3424 - val_acc: 0.5200
Epoch 64/200
1/1 [==============================] - 0s 161ms/step - loss: 0.0942 - acc: 0.7071 - val_loss: 0.3420 - val_acc: 0.5240
Epoch 65/200
1/1 [==============================] - 0s 160ms/step - loss: 0.0937 - acc: 0.7143 - val_loss: 0.3414 - val_acc: 0.5240
Epoch 66/200
1/1 [==============================] - 0s 164ms/step - loss: 0.0940 - acc: 0.6857 - val_loss: 0.3410 - val_acc: 0.5280
Epoch 67/200
1/1 [==============================] - 0s 172ms/step - loss: 0.0931 - acc: 0.7214 - val_loss: 0.3405 - val_acc: 0.5320
Epoch 68/200
1/1 [==============================] - 0s 168ms/step - loss: 0.0932 - acc: 0.7071 - val_loss: 0.3400 - val_acc: 0.5300
Epoch 69/200
1/1 [==============================] - 0s 184ms/step - loss: 0.0930 - acc: 0.6857 - val_loss: 0.3394 - val_acc: 0.5300
Epoch 70/200
1/1 [==============================] - 0s 176ms/step - loss: 0.0936 - acc: 0

Epoch 125/200
1/1 [==============================] - 0s 160ms/step - loss: 0.0827 - acc: 0.8286 - val_loss: 0.3089 - val_acc: 0.6480
Epoch 126/200
1/1 [==============================] - 0s 156ms/step - loss: 0.0829 - acc: 0.7857 - val_loss: 0.3082 - val_acc: 0.6500
Epoch 127/200
1/1 [==============================] - 0s 163ms/step - loss: 0.0830 - acc: 0.7714 - val_loss: 0.3074 - val_acc: 0.6560
Epoch 128/200
1/1 [==============================] - 0s 168ms/step - loss: 0.0848 - acc: 0.8000 - val_loss: 0.3068 - val_acc: 0.6560
Epoch 129/200
1/1 [==============================] - 0s 165ms/step - loss: 0.0837 - acc: 0.8000 - val_loss: 0.3061 - val_acc: 0.6680
Epoch 130/200
1/1 [==============================] - 0s 164ms/step - loss: 0.0831 - acc: 0.8357 - val_loss: 0.3054 - val_acc: 0.6720
Epoch 131/200
1/1 [==============================] - 0s 168ms/step - loss: 0.0813 - acc: 0.8571 - val_loss: 0.3046 - val_acc: 0.6760
Epoch 132/200
1/1 [==============================] - 0s 169ms/step - 

1/1 [==============================] - 0s 159ms/step - loss: 0.0747 - acc: 0.8714 - val_loss: 0.2767 - val_acc: 0.7480
Epoch 187/200
1/1 [==============================] - 0s 159ms/step - loss: 0.0750 - acc: 0.8714 - val_loss: 0.2763 - val_acc: 0.7460
Epoch 188/200
1/1 [==============================] - 0s 160ms/step - loss: 0.0766 - acc: 0.8500 - val_loss: 0.2761 - val_acc: 0.7480
Epoch 189/200
1/1 [==============================] - 0s 167ms/step - loss: 0.0741 - acc: 0.8571 - val_loss: 0.2760 - val_acc: 0.7460
Epoch 190/200
1/1 [==============================] - 0s 172ms/step - loss: 0.0761 - acc: 0.8714 - val_loss: 0.2759 - val_acc: 0.7420
Epoch 191/200
1/1 [==============================] - 0s 162ms/step - loss: 0.0735 - acc: 0.9214 - val_loss: 0.2758 - val_acc: 0.7420
Epoch 192/200
1/1 [==============================] - 0s 163ms/step - loss: 0.0739 - acc: 0.8929 - val_loss: 0.2754 - val_acc: 0.7420
Epoch 193/200
1/1 [==============================] - 0s 163ms/step - loss: 0.0738 -